# getting data in... to the graph

In [1]:
import tensorflow as tf

graph = tf.get_default_graph()
session = tf.Session()

In [2]:
graph.get_operations()

[]

In [3]:
zero_d = tf.constant(4)

In [4]:
graph.get_operations()

[<tf.Operation 'Const' type=Const>]

In [5]:
session.run(zero_d - 1)

3

In [6]:
graph.get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'sub/y' type=Const>,
 <tf.Operation 'sub' type=Sub>]

In [7]:
session.run(zero_d, feed_dict={zero_d: 9})

array(9, dtype=int32)

In [8]:
graph.get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'sub/y' type=Const>,
 <tf.Operation 'sub' type=Sub>]

Disclaimer: This example gleefully uses `tf.constant()` and not `tf.placeholder()`. Really if you plan to feed data in, you should use `tf.placeholder()`. Also you likely want to provide more useful `name` values for things, etc.

In [9]:
one_d = tf.constant([1, 1])

In [10]:
session.run(one_d + [2, 2])

array([3, 3], dtype=int32)

In [11]:
import numpy as np

two_d = tf.constant(np.array([[2, 2], [2, 2]]))

In [12]:
session.run(two_d + [[1, 1], [1, 1]])

array([[3, 3],
       [3, 3]])

In [13]:
session.run(two_d, feed_dict={two_d: [[9, 9], [9, 9]]})

array([[9, 9],
       [9, 9]])

---

# getting data in... TFRecords

In [ ]:
# This is how Caffe might store an image and label in LMDB.

import caffe
import lmdb

image = np.ones((1, 28, 28))  # not a meaningful image
label = 9  # for a classification problem

datum = caffe.io.array_to_datum(arr=image, label=label)
datum_str = datum.SerializeToString()

env = lmdb.open('lmdb_data')
txn = env.begin(write=True)
txn.put(key='my datum', value=datum_str)

cur = txn.cursor()
same_datum_str = cur.get('my datum')

same_datum = caffe.proto.caffe_pb2.Datum().FromString(same_datum_str)
same_image = caffe.io.datum_to_array(same_datum)
same_label = datum.label

 * Caffe `datum` -> TensorFlow `Example`
 * LMDB database -> TensorFlow `TFRecords` file

In [14]:
# Python dict:
my_dict = {'features' : {
    'my_ints': [5, 6],
    'my_float': [2.7],
    'my_bytes': ['data']
}}

In [15]:
# TFRecords `Example`:
my_example = tf.train.Example(features=tf.train.Features(feature={
    'my_ints': tf.train.Feature(int64_list=tf.train.Int64List(value=[5, 6])),
    'my_float': tf.train.Feature(float_list=tf.train.FloatList(value=[2.7])),
    'my_bytes': tf.train.Feature(bytes_list=tf.train.BytesList(value=['data']))
}))

In [16]:
image = np.ones((1, 28, 28), dtype=np.uint8)  # not a meaningful image
label = 9  # for a classification problem

image_bytes = image.tostring()
image_shape = image.shape

my_example = tf.train.Example(features=tf.train.Features(feature={
    'image_bytes': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
    'image_shape': tf.train.Feature(int64_list=tf.train.Int64List(value=image_shape)),
    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
}))

In [17]:
my_example_str = my_example.SerializeToString()
with tf.python_io.TFRecordWriter('my_example.tfrecords') as writer:
    writer.write(my_example_str)

reader = tf.python_io.tf_record_iterator('my_example.tfrecords')
those_examples = [tf.train.Example().FromString(example_str)
                  for example_str in reader]

In [18]:
same_example = those_examples[0]
same_image_bytes = same_example.features.feature['image_bytes'].bytes_list.value[0]
same_image_shape = list(same_example.features.feature['image_shape'].int64_list.value)
same_label = same_example.features.feature['label'].int64_list.value[0]
same_image = np.fromstring(same_image_bytes, dtype=np.uint8)
same_image.shape = same_image_shape

In [19]:
same_label

9L

---

# Exploring

 * [Python quick reference](http://www.dataschool.io/python-quick-reference/)
 * `dir()` and `help()`
 * Using TensorFlow documentation

In [20]:
dir(tf)

['AggregationMethod',
 'Assert',
 'AttrValue',
 'COMPILER_VERSION',
 'ConditionalAccumulator',
 'ConditionalAccumulatorBase',
 'ConfigProto',
 'DType',
 'DeviceSpec',
 'Dimension',
 'Event',
 'FIFOQueue',
 'FixedLenFeature',
 'FixedLenSequenceFeature',
 'FixedLengthRecordReader',
 'GIT_VERSION',
 'GPUOptions',
 'GRAPH_DEF_VERSION',
 'GRAPH_DEF_VERSION_MIN_CONSUMER',
 'GRAPH_DEF_VERSION_MIN_PRODUCER',
 'Graph',
 'GraphDef',
 'GraphKeys',
 'GraphOptions',
 'HistogramProto',
 'IdentityReader',
 'IndexedSlices',
 'InteractiveSession',
 'LogMessage',
 'NameAttrList',
 'NoGradient',
 'NodeDef',
 'NotDifferentiable',
 'OpError',
 'Operation',
 'OptimizerOptions',
 'PaddingFIFOQueue',
 'Print',
 'PriorityQueue',
 'QUANTIZED_DTYPES',
 'QueueBase',
 'RandomShuffleQueue',
 'ReaderBase',
 'RegisterGradient',
 'RunMetadata',
 'RunOptions',
 'Session',
 'SessionLog',
 'SparseConditionalAccumulator',
 'SparseFeature',
 'SparseTensor',
 'SparseTensorValue',
 'Summary',
 'TFRecordReader',
 'Tensor',
 '

In [21]:
help(tf.TFRecordReader)

Help on class TFRecordReader in module tensorflow.python.ops.io_ops:

class TFRecordReader(ReaderBase)
 |  A Reader that outputs the records from a TFRecords file.
 |  
 |  See ReaderBase for supported methods.
 |  
 |  Method resolution order:
 |      TFRecordReader
 |      ReaderBase
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name=None, options=None)
 |      Create a TFRecordReader.
 |      
 |      Args:
 |        name: A name for the operation (optional).
 |        options: A TFRecordOptions object (optional).
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ReaderBase:
 |  
 |  num_records_produced(self, name=None)
 |      Returns the number of records this reader has produced.
 |      
 |      This is the same as the number of Read executions that have
 |      succeeded.
 |      
 |      Args:
 |        name: A name for the operation (optional).
 |      
 |      Returns:
 |        An 

 * Search: https://www.tensorflow.org/s/results/?q=TFRecordReader
 * Docs: https://www.tensorflow.org/api_docs/python/tf/TFRecordReader
 * Source: https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/python/ops/io_ops.py#L413